# Test Notebook

For Debugging the Individual Chains

## Setup

In [1]:
%load_ext dotenv
%dotenv ../.env

In [2]:
import os
assert "HUGGINGFACEHUB_API_TOKEN" in os.environ

In [1]:
from langchain_openai import ChatOpenAI

model_kwargs={
    "model": "meta-llama/Llama-3.1-8B-Instruct-FP8",
    "base_url": "https://huggingface.co/api/integrations/dgx/v1",
    "api_key": os.environ["HUGGINGFACEHUB_API_TOKEN"],
}
formatter_model_kwargs={
    "model": "tgi",
    "base_url": "http://kriton.philosophie.kit.edu:8080/v1/",
    "api_key": "NONE",
}

chat_model = ChatOpenAI(**model_kwargs)
formatter_model = ChatOpenAI(**formatter_model_kwargs)


In [2]:
# test API
from langchain_core.prompts import ChatPromptTemplate
chain = ChatPromptTemplate.from_messages([("user","Tell me a joke about {x}! (Format as JSON)")]) | chat_model.bind(response_format= {"type":"json_object"})
chain.invoke({"x":"kids"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a joke about kids!', additional_kwargs={}, response_metadata={})])

# Test Classifier

In [4]:
from syncialo.chains.classifier import ClassificationResult, classify

await classify(["""In a debate about veganism, it is maintained:

PREMISE: The pace of the evolution of consciousness, which is a product of millions of years of evolution on Earth, is unlikely to be affected by human space exploration.

HYPOTHESIS: Consciousness is a product of millions of years of evolution on Earth, and the pace of its evolution is likely to be unaffected by human space exploration, which is a relatively recent phenomenon.
""", ], labels=["entailment", "independence", "contradiction"])

[ClassificationResult(sequence='In a debate about veganism, it is maintained:\n\nPREMISE: The pace of the evolution of consciousness, which is a product of millions of years of evolution on Earth, is unlikely to be affected by human space exploration.\n\nHYPOTHESIS: Consciousness is a product of millions of years of evolution on Earth, and the pace of its evolution is likely to be unaffected by human space exploration, which is a relatively recent phenomenon.\n', labels=['entailment', 'independence', 'contradiction'], scores=[0.6388322710990906, 0.1976313441991806, 0.16353647410869598])]

# Test VectorStore

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_core.documents.base import Document


embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"), model_name="sentence-transformers/all-MiniLM-l6-v2"
)

_documents = [
    {"claim": "Eating animals is bad for the environment.", "uid": "1638-1772-1"},
    {"claim": "Wild boars are a threat to the environment.", "uid": "1638-1772-2"},
    {"claim": "Wildlife is the most important thing in the world.", "uid": "1638-1772-3"},
]

vector_store = FAISS.from_documents(embedding=embeddings, documents=[Document(page_content=_d["claim"], metadata={"uid":_d["uid"]}) for _d in _documents])




In [11]:
vector_store.search("I cherish wildlife.", search_type="similarity", k=2)[0].metadata

{'uid': '1638-1772-3'}

In [12]:
vector_store.add_documents([Document(page_content="I cherish wildlife.", id="1638-1772-4")])

['1638-1772-4']

# Test Debate Design Chains

In [5]:
import random 

with open('../data/universal_tags.txt') as file:
    tags = [line.rstrip() for line in file]
testtags = random.sample(tags, 8)
print(f"Loaded {len(tags)} tags. Choose: {testtags}.")


Loaded 280 tags. Choose: ['Refugees', 'Censorship', 'Europe', 'Copyright', 'War', 'Europa', 'Poverty', 'Pandemic'].


In [8]:
from syncialo.chains.debate_design import SuggestTopicsChain

chain = SuggestTopicsChain.build(chat_model, llm_formatting=formatter_model)

In [9]:
import ujson

from langchain.globals import set_debug

set_debug(False)

output = chain.invoke({
    "tags": testtags,
    "debates_per_tag_cluster": 5
})

print(ujson.dumps(output, indent=2))

[
  {
    "idx": "1",
    "topic": "Should European countries prioritize national security over refugee rights during a pandemic?"
  },
  {
    "idx": "2",
    "topic": "Resolved, that censorship of online content is necessary to combat the spread of misinformation during a pandemic in Europe."
  },
  {
    "idx": "3",
    "topic": "Does the enforcement of copyright laws hinder the accessibility of educational resources for impoverished refugees in Europe?"
  },
  {
    "idx": "4",
    "topic": "Is military intervention justified in resolving conflicts that lead to refugee crises and poverty in Europe?"
  },
  {
    "idx": "5",
    "topic": "Should the European Union prioritize economic aid over military spending in response to the humanitarian crises caused by war and pandemic?"
  }
]


In [9]:
from syncialo.chains.debate_design import SuggestMotionChain

from langchain.globals import set_debug

set_debug(False)

chain_motion = SuggestMotionChain.build(chat_model)

output_motion = chain_motion.invoke({
    "topic": "How could changes in artistic expression reflect and influence significant social issues such as poverty?",
    "tags": testtags
})

print(output_motion)

{'motion': 'Increased government funding for the arts is a necessary step to address poverty in the United States.', 'title': 'Arts for the Poor'}


# Argumentation Chains

In [6]:
from syncialo.chains.argumentation import IdentifyPremisesChain, Valence

chain_premises = IdentifyPremisesChain.build(chat_model, llm_formatting=formatter_model)

output = chain_premises.invoke({
    "argument": "Meat contains major nutrients.",
    "conclusion": "It's okay to eat meat.",
    "valence": Valence.PRO
})

print(output)


['Meat contains major nutrients.', 'Major nutrients are essential for human health.', 'Foods that contain essential nutrients are okay to eat.', 'The nutritional benefits of meat outweigh any potential drawbacks.', 'Human dietary needs should be prioritized when evaluating food choices.']


In [4]:
from syncialo.chains.argumentation import RankPropsByPlausibilityChain

chain_rank = RankPropsByPlausibilityChain.build(chat_model, llm_formatting=formatter_model)

output = chain_rank.invoke({
    "premises": [
        'Meat contains major nutrients',
        'Consuming major nutrients is important for overall health and well-being', 
        'Eating meat provides major nutrients', 
        'Maintaining good health through proper nutrition is desirable', 
        'The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns'
    ],
    "tags": ["climate change", "animals", "health"],
    "persona": "A foodtruck owner in Chicago."
})

print(output)

[1, 0, 3, 2, 4]


In [9]:
from langchain.globals import set_debug
from syncialo.chains.argumentation import GenSupportingArgumentChain

set_debug(True)

chain_support = GenSupportingArgumentChain.build(chat_model, llm_formatting=formatter_model)

output = chain_support.invoke({
    "premises": [
        'Meat contains major nutrients',
        'Consuming major nutrients is important for overall health and well-being', 
        'Eating meat provides major nutrients', 
        'Maintaining good health through proper nutrition is desirable', 
        'The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns'
    ],
    "tags_pro": ["climate change", "animals", "health"],
    "persona": "A foodtruck owner in Chicago.",
    "ranking": [0,1,2,3,4],
    "n": 4
})

print(output)


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "premises": [
    "Meat contains major nutrients",
    "Consuming major nutrients is important for overall health and well-being",
    "Eating meat provides major nutrients",
    "Maintaining good health through proper nutrition is desirable",
    "The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns"
  ],
  "tags_pro": [
    "climate change",
    "animals",
    "health"
  ],
  "persona": "A foodtruck owner in Chicago.",
  "ranking": [
    0,
    1,
    2,
    3,
    4
  ],
  "n": 4
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<target_idx>] Entering Chain run with input:
{
  "premises": [
    "Meat contains major nutrients",
    "Consuming major nutrients is important for overall health and well-being",
    "Eating meat provides major nutrients",
    "Maintaining good health through proper nutrition is desirable",
    "The benefi

In [10]:
from syncialo.chains.argumentation import GenAttackingArgumentChain

chain_support = GenAttackingArgumentChain.build(chat_model, llm_formatting=formatter_model)

output = chain_support.invoke({
    "premises": [
        'Meat contains major nutrients',
        'Consuming major nutrients is important for overall health and well-being', 
        'Eating meat provides major nutrients', 
        'Maintaining good health through proper nutrition is desirable', 
        'The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns'
    ],
    "tags_con": ["climate change", "books", "India"],
    "persona": "A foodtruck owner in Chicago.",
    "ranking": [0,1,2,3,4],
    "n": 4
})

print(output)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "premises": [
    "Meat contains major nutrients",
    "Consuming major nutrients is important for overall health and well-being",
    "Eating meat provides major nutrients",
    "Maintaining good health through proper nutrition is desirable",
    "The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns"
  ],
  "tags_con": [
    "climate change",
    "books",
    "India"
  ],
  "persona": "A foodtruck owner in Chicago.",
  "ranking": [
    0,
    1,
    2,
    3,
    4
  ],
  "n": 4
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<target_idx>] Entering Chain run with input:
{
  "premises": [
    "Meat contains major nutrients",
    "Consuming major nutrients is important for overall health and well-being",
    "Eating meat provides major nutrients",
    "Maintaining good health through proper nutrition is desirable",
    "The benefits 

In [11]:
from syncialo.chains.argumentation import ArgumentModel, Valence
from syncialo.chains.argumentation import SelectMostSalientChain

assert isinstance(output[0], ArgumentModel)
print(len(output))
print(output)

chain_salient = SelectMostSalientChain.build(chat_model, llm_formatting=formatter_model)

s_output = chain_salient.invoke({
    "args": output,
    "k": 2,
    "conclusion": "Meat contains major nutrients.",
    "valence": Valence.CON,
})

print(len(s_output))
print(s_output)


4
[ArgumentModel(label='Climate Concerns', claim='The production and consumption of meat contribute significantly to greenhouse gas emissions, which outweigh the nutritional benefits of meat consumption.', target_idx=4, valence=<Valence.PRO: 'PRO'>), ArgumentModel(label='Indian Insights', claim='Traditional Indian diets, which are often vegetarian or vegan, have been associated with lower rates of chronic diseases, challenging the idea that meat consumption is necessary for overall health.', target_idx=4, valence=<Valence.PRO: 'PRO'>), ArgumentModel(label='Resource Inefficiency', claim='Meat production requires significantly more water, land, and energy than plant-based food production, making it a resource-inefficient choice for obtaining essential nutrients.', target_idx=4, valence=<Valence.PRO: 'PRO'>), ArgumentModel(label='Booked on Saturated Fat', claim='Consuming meat can lead to an excessive intake of saturated fat, which can increase the risk of heart disease and other health p

In [12]:
from syncialo.chains.argumentation import GenerateProAndConChain

chain_support = GenerateProAndConChain.build(chat_model, llm_formatting=formatter_model)

output = chain_support.invoke({
    "premises": [
        'Meat contains major nutrients',
        'Consuming major nutrients is important for overall health and well-being', 
        'Eating meat provides major nutrients', 
        'Maintaining good health through proper nutrition is desirable', 
        'The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns'
    ],
    "tags": ["climate change", "animals", "health"],
    "tags_universal": ["climate change", "books", "India", "Trees"],
    "tags_per_cluster": 3,
    "persona": "A foodtruck owner in Chicago.",
    "ranking": [0,1,2,3,4],
    "n": 2
})

print(output)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "premises": [
    "Meat contains major nutrients",
    "Consuming major nutrients is important for overall health and well-being",
    "Eating meat provides major nutrients",
    "Maintaining good health through proper nutrition is desirable",
    "The benefits of consuming meat, including the provision of major nutrients, outweigh any potential drawbacks or concerns"
  ],
  "tags": [
    "climate change",
    "animals",
    "health"
  ],
  "tags_universal": [
    "climate change",
    "books",
    "India",
    "Trees"
  ],
  "tags_per_cluster": 3,
  "persona": "A foodtruck owner in Chicago.",
  "ranking": [
    0,
    1,
    2,
    3,
    4
  ],
  "n": 2
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<persona,tags_pro,tags_con>] Entering Chain run with input:
{
  "premises": [
    "Meat contains major nutrients",
    "Consuming major nutrients is important for overall health and well-being",
    "

## Tags

In [5]:
import os

UNIVERSAL_TAGS_PATH = "../data/universal_tags.txt"
EVAL_TAGS_PATH = "../data/eval_tags.txt"
TEST_TAGS_PATH = "../data/test_tags.txt"

tags_universal = None
tags_eval = None
tags_test = None

if (
    os.path.exists(UNIVERSAL_TAGS_PATH) and
    os.path.exists(EVAL_TAGS_PATH) and
    os.path.exists(TEST_TAGS_PATH)
):
    with open(UNIVERSAL_TAGS_PATH) as file:
        tags_universal = [line.rstrip() for line in file]
        print(f"Loaded {len(tags_universal)} universal tags.")
    with open(EVAL_TAGS_PATH) as file:
        tags_eval = [line.rstrip() for line in file]
        print(f"Loaded {len(tags_eval)} eval tags.")
    with open(TEST_TAGS_PATH) as file:
        tags_test = [line.rstrip() for line in file]
        print(f"Loaded {len(tags_test)} test tags.")
else:
    raise Exception("Failed to load tags; data files missing.")

Loaded 280 universal tags.
Loaded 10 eval tags.
Loaded 10 test tags.


# Test DebateBuilder

In [6]:
from langchain.globals import set_debug

set_debug(False)

In [7]:
from syncialo.debate_builder import DebateBuilder

debateBuilder = DebateBuilder(
    model=chat_model,
    tags_universal=tags_universal,
    tags_per_cluster=6,
)


/Users/ggbetz/git/syncIALO/.hatch/syncialo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
built_debate = await debateBuilder.build_debate(
    motion="Museums should be free for everyone.",
    topic="Should we pay for culture?",
    tag_cluster=tags_universal[:6],
    degree_config=[2,2,0],
)

2024-10-24 19:22:05.961 | DEBUG    | syncialo.debate_builder:init_vector_store:79 - Initializing vector store for duplicate detection.
2024-10-24 19:22:06.630 | DEBUG    | syncialo.debate_builder:build_subtree:180 - Processing at depth 0
2024-10-24 19:22:06.631 | DEBUG    | syncialo.debate_builder:build_subtree:181 - Degree = 2
2024-10-24 19:22:06.632 | DEBUG    | syncialo.debate_builder:build_subtree:182 - Target reason claim: Museums should be free for everyone.
2024-10-24 19:23:06.888 | DEBUG    | syncialo.debate_builder:build_subtree:180 - Processing at depth 1
2024-10-24 19:23:06.890 | DEBUG    | syncialo.debate_builder:build_subtree:181 - Degree = 2
2024-10-24 19:23:06.890 | DEBUG    | syncialo.debate_builder:build_subtree:182 - Target reason claim: Museums house cultural and historical ar
2024-10-24 19:24:47.583 | INFO     | syncialo.debate_builder:get_equivalent:150 - Found equivalent node for 'Museums preserve and showcase historical artifacts that are significant not only to 

In [11]:
# pretty print the networkx graph `built_debate`

import pprint
pprint.pprint(list(built_debate.nodes(data=True)))
pprint.pprint(list(built_debate.edges(data=True)))



[('2dc0257f-1763-4d46-830d-d58232da72ee',
  {'claim': 'Museums should be free for everyone.', 'label': ''}),
 ('3bdc7b6c-1e9c-4450-8a04-862487f06a7a',
  {'claim': 'Museums house cultural and historical artifacts that belong to '
            'the collective heritage of humanity, and making them free ensures '
            'that everyone can access and appreciate their shared history.',
   'label': 'Cultural Heritage for All',
   'premises': ['Museums house cultural and historical artifacts that belong '
                'to the collective heritage of humanity.',
                'Making museums free ensures that everyone can access and '
                'appreciate their shared history.',
                'Everyone has a right to access and appreciate their shared '
                'cultural and historical heritage.',
                'Financial barriers can prevent people from accessing museums '
                'and appreciating their shared history.',
                'Ensuring universal a

In [16]:
import networkx as nx

from syncialo.chains.argumentation import Valence

data = nx.node_link_data(built_debate)
#pprint.pprint(data)

def get_parents(data, node_id, valence):
    parents = []
    for edge in data["links"]:
        if edge["target"] == node_id and edge["valence"] == valence.value:
            parents.append(edge["source"])
    return parents

root_id = data["nodes"][0]["id"]

def print_argdown_line(data, node_id, valence=None, level=0):
    node = next(n for n in data["nodes"] if n["id"]==node_id)
    indent = level * "  "
    marker = "+ " if valence == Valence.PRO else "- " if valence == Valence.CON else ""
    print(f"{indent}{marker}[{node.get('label')}]: {node['claim']}")
    for pro in get_parents(data, node_id, Valence.PRO):
        print_argdown_line(data, pro, Valence.PRO, level + 1)
    for con in get_parents(data, node_id, Valence.CON):
        print_argdown_line(data, con, Valence.CON, level + 1)

print_argdown_line(data, root_id)


[]: Museums should be free for everyone.
  + [Cultural Heritage for All]: Museums house cultural and historical artifacts that belong to the collective heritage of humanity, and making them free ensures that everyone can access and appreciate their shared history.
    + [Cultural Significance]: Museums preserve and showcase historical artifacts that are significant not only to their countries of origin but also to the global community, as evidenced by the popularity of international exhibitions and collaborations between museums worldwide.
    + [Educational Value]: Museums house cultural and historical artifacts that serve as valuable teaching tools, allowing students and researchers from diverse backgrounds to engage with and learn from the collective heritage of humanity, as seen in the numerous educational programs and partnerships between museums and academic institutions in the USA, UK, and Asia.
    - [Funding Fiasco]: Making museums free would require significant funding, which

In [20]:
import ujson

def pprint_debate(path):
    with open(path) as file:
        data = ujson.load(file)
        print_argdown_line(data, data["nodes"][0]["id"])

corpus="synthetic_corpus-TEST-002"
split="train"
debate_id="0003"
pprint_debate(f"../output/{corpus}/{split}/debate-{split}-{debate_id}/node_link_data-debate-{split}-{debate_id}.json")

["Face Off Our Rights"]: Facial recognition technology has no place in law enforcement due to its inherent threat to individual freedoms.
  + [Error Prone]: Facial recognition technology has been proven to have high error rates, especially among certain demographics, which can lead to wrongful arrests and convictions, further eroding trust in law enforcement and threatening individual freedoms.
    + [Biased Code]: Facial recognition technology is often trained on biased datasets, which can result in higher error rates for demographics that are underrepresented or misrepresented in the training data.
    - [Pragmatic Balance]: Prioritizing individual freedoms over all other concerns can lead to societal chaos and undermine the greater good, requiring a balanced approach that weighs individual rights against collective well-being.
  + [Surveillance State]: The use of facial recognition technology by law enforcement can create a chilling effect on free speech and assembly, as people may 